In [98]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [121]:
tags = ["bbfm", "blu", "dbmc", "dbu"]
path="data2/subms2/"
tm=pd.read_csv(path+"LR_train_" + tags[0] + ".csv")
for t in tags[1:]:
    tm_t=pd.read_csv(path+"LR_train_" + t + ".csv")
    tm = pd.concat([tm, tm_t],  axis=1, sort="False")
tm.columns=["id", "bbfm0", "bbfm1", "id1", "blu0", "blu1", "id2", "dbmc0", "dbmc1", "id3", "dbu0", "dbu1"]
tm = tm.drop(columns = ["id", "id1", "id2", "id3"])
tm.info()

tt=pd.read_csv(path+"LR_test_" + tags[0] + ".csv")
for t in tags[1:]:
    tt_t=pd.read_csv(path+"LR_test_" + t + ".csv")
    tt = pd.concat([tt, tt_t],  axis=1, sort="False")
tt.columns=["bbfm0", "bbfm1", "id1", "blu0", "blu1", "id2", "dbmc0", "dbmc1", "id3", "dbu0", "dbu1", "id"]
tt = tt.drop(columns = ["id", "id1", "id2", "id3"])
tt.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7577 entries, 0 to 7576
Data columns (total 8 columns):
bbfm0    7577 non-null float64
bbfm1    7577 non-null float64
blu0     7577 non-null float64
blu1     7577 non-null float64
dbmc0    7577 non-null float64
dbmc1    7577 non-null float64
dbu0     7577 non-null float64
dbu1     7577 non-null float64
dtypes: float64(8)
memory usage: 532.8 KB


,bbfm0,bbfm1,blu0,blu1,dbmc0,dbmc1,dbu0,dbu1
0,0.189700,0.810300,0.798157,0.201843,0.728796,0.271204,0.657848,0.342152
1,0.815193,0.184807,0.305174,0.694826,0.526895,0.473105,0.479307,0.520693
2,0.693118,0.306882,0.478265,0.521735,0.758916,0.241084,0.327140,0.672860
3,0.465923,0.534077,0.184017,0.815983,0.690621,0.309379,0.059447,0.940553
4,0.050267,0.949733,0.083352,0.916648,0.012043,0.987957,0.003737,0.996263


In [129]:
df = pd.read_csv('data_proc2.csv')
labels = df["target"][:7577]
labels.shape

(7577,)

In [132]:
train_features, test_features, train_labels, test_labels = train_test_split(tm, labels, shuffle="True")

In [140]:
parameters = {'C': np.linspace(0.0001, 100, 20), 'solver' : ['newton-cg', 'sag', 'saga', 'lbfgs' ]}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947, 'solver': 'newton-cg'}
best scrores:  0.8157323448164803


In [141]:
lr_clf = LogisticRegression( C = grid_search.best_params_['C'])
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=5.263252631578947, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [142]:
lr_clf.score(test_features, test_labels)

0.8258575197889182

In [143]:
w=lr_clf.coef_
w

array([[-0.2969271 ,  0.33775521, -0.86616464,  0.90699276, -0.33149989,
         0.372328  , -1.59911361,  1.63994172]])

In [170]:
lr_clf.fit(tm, labels)
Y_pred = lr_clf.predict(tt)
Y_pred

array([[0.71128475, 0.28871525],
       [0.42397206, 0.57602794],
       [0.40221883, 0.59778117],
       ...,
       [0.0682367 , 0.9317633 ],
       [0.18724234, 0.81275766],
       [0.24743156, 0.75256844]])

In [171]:
df["id"].shape

(3263,)

(3263, 3)

In [174]:
import time
submission = pd.DataFrame({"id": df["id"][7577:], "target": Y_pred}, dtype = "int")
fname = '' + time.ctime() + 'submission.csv'
submission.to_csv(fname, index=False, header="id,target")

Exception: Data must be 1-dimensional

#Следующая серия

In [157]:
files = ["id","SVCdbu.csv", "LRblu.csv", "SVCblu.csv"]
path="data2/subms/"


In [165]:
df=pd.read_csv(path + files[1])

for f in files[2:]:
    df = pd.concat([df, pd.read_csv(path + f)],  axis=1, sort="False")
df.columns = ["id", files[1], "id1", files[2], "id2", files[3]]
df= df.drop(["id1", "id2"], axis=1)
df.head()

,id,SVCdbu.csv,LRblu.csv,SVCblu.csv
0,0,0,0,0
1,2,1,1,1
2,3,1,1,1
3,9,1,1,1
4,11,1,1,1


In [166]:
df[files[1]] + df[files[2]] + df[files[3]] 

0       0
1       3
2       3
3       3
4       3
       ..
3258    0
3259    3
3260    3
3261    3
3262    3
Length: 3263, dtype: int64

In [167]:
import math
df["final"] = df[files[1]] + df[files[2]] + df[files[3]] 
df["mark"] = df["final"].apply(lambda x: math.ceil(x/3 - 0.5))

In [168]:
df.head(10)

,id,SVCdbu.csv,LRblu.csv,SVCblu.csv,final,mark
0,0,0,0,0,0,0
1,2,1,1,1,3,1
2,3,1,1,1,3,1
3,9,1,1,1,3,1
4,11,1,1,1,3,1
5,12,1,0,0,1,0
6,21,0,0,0,0,0
7,22,0,0,0,0,0
8,27,0,0,0,0,0
9,29,0,0,0,0,0


In [ ]:
df.shape

In [169]:
import time
submission = pd.DataFrame({"id": df["id"], "target": df["mark"] }, dtype = "int")
fname = '' + time.ctime() + 'submission.csv'
submission.to_csv(fname, index=False, header="id,target")